In [2]:
from datasets import load_dataset
import requests
import ciso8601
import time
import json
from datetime import datetime

In [ ]:
ds = load_dataset("codeparrot/github-code", streaming=True, split='train', languages=["Python"])

In [8]:
def construct_feature_set(code_entry) -> dict[str:int]:
    features = dict();
    user_repo_name = code_entry['repo_name'].split('/')
    owner = user_repo_name[0]
    repo = "".join(user_repo_name[1:])
    repo_response = requests.get(f"https://api.github.com/repos/{owner}/{repo}")
    content = json.loads(repo_response.text)
    features["num_stars"] = content.get("stargazers_count", 0)
    features["num_forks"] = content.get("forks_count", 0)
    features["num_watchers"] = content.get("watchers_count", 0)
    features["num_open_issues"] = content.get("open_issues_count", 0)
    parsed_datetime = ciso8601.parse_datetime(content.get("created_at", datetime.now().isoformat()))
    timestamp = time.mktime(parsed_datetime.timetuple())
    features["created_at"] = timestamp
    return features

In [7]:
# THE BELOW IS FOR PYTHON FILES (LEVERAGES INDENTATION RULES)
def construct_list_of_functions(code_entry) -> list[str]:
    raw_code = code_entry["code"]
    lines = raw_code.split('\n')
    start = -1
    functions = []
    amnt_tabs = 0
    for i in range(len(lines)):
        # disregard empty lines (prune trailing whitespace later)
        if (start != -1 and len(lines[i]) > 0):
            amnt_tabs_new = len(lines[i].rstrip()) - len(lines[i].strip())
            if amnt_tabs_new <= amnt_tabs:
                functions.append(("\n".join(lines[start:i])).strip())
                start = -1
        elif lines[i].lstrip().startswith("def "):
            start = i
            amnt_tabs = len(lines[i].rstrip()) - len(lines[i].strip())
    return functions


In [39]:
MAX_AMNT_CODE_ENTRIES = 100
augmented_code_entry_data = []
for i in range(MAX_AMNT_CODE_ENTRIES):
    code_entry = next(iterable_ds)
    features_set = construct_feature_set(code_entry)
    # functions = construct_list_of_functions(code_entry["code"])
    data = dict()
    data["features"] = []
    data["snippets"] = []
    augmented_code_entry_data.append(data)

In [9]:
def augment_code_entry(entry):
    repo_features = construct_feature_set(entry)
    entry["reputation_features"] = repo_features
    if entry["language"] != "Python":
        return entry
    entry["functions"] = construct_list_of_functions(entry)
    return entry

In [ ]:
augmented_dataset = ds.map(augment_code_entry).to_pandas()

In [32]:
dataset_heads = augmented_dataset.take(4)
arr = list(dataset_heads)

In [35]:
print('\n'.join(arr[0]["functions"]))

def __init__(self, allow=None, disallow=None, secure=True, *args, **kwargs):
		super(TemplateField, self).__init__(*args, **kwargs)
		self.validators.append(TemplateValidator(allow, disallow, secure))
def __init__(self, field):
		self.field = field
def __get__(self, instance, owner):
		if instance is None:
			raise AttributeError # ?
def __set__(self, instance, value):
		instance.__dict__[self.field.name] = value
		setattr(instance, self.field.attname, json.dumps(value))
def __delete__(self, instance):
		del(instance.__dict__[self.field.name])
		setattr(instance, self.field.attname, json.dumps(None))
def get_attname(self):
		return "%s_json" % self.name
def contribute_to_class(self, cls, name):
		super(JSONField, self).contribute_to_class(cls, name)
		setattr(cls, name, JSONDescriptor(self))
		models.signals.pre_init.connect(self.fix_init_kwarg, sender=cls)
def fix_init_kwarg(self, sender, args, kwargs, **signal_kwargs):
		# Anything passed in as self.name is assumed to come from a ser